In [1]:
import time
from tqdm import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf
from connect.connect_mondodb import ConnectMongoDB

2022-08-30 14:14:27.740972: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-30 14:14:27.741083: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
inicio = time.time()
conexao = ConnectMongoDB(con='mongodb://127.0.0.1:27017/')
fim = time.time()
print(fim - inicio)
collections = conexao.show_collection('levi')

0.01145172119140625


In [3]:
collections

['RQ_Q',
 'RQ',
 'metrics_RC',
 'RQRQ_RQ',
 'RQRQ_RQ_Q',
 'metrics_RQ',
 'RQ_RQ',
 'metrics_R_RQ_RQ',
 'metrics_RQ_Q',
 'metrics_R_RQ',
 'metrics_RQ_RQ',
 'R_RQ_RQ',
 'metrics_RQRQ_RQ',
 'RC',
 'R_RQ']

In [4]:
rotulo_collection = [
    collection for collection 
    in collections if 
    collection.startswith('metrics_') == False
]

In [5]:
rotulo_collection

['RQ_Q', 'RQ', 'RQRQ_RQ', 'RQRQ_RQ_Q', 'RQ_RQ', 'R_RQ_RQ', 'RC', 'R_RQ']

In [7]:
# usando numpy.column_stack
def extend_ZreZim(dataframe):
    z_re = []
    z_im = []
    for real, img in zip(dataframe['Z_re'].values, dataframe['Z_im']):
        z_re.extend(real)
        z_im.extend(img)
    return np.column_stack((z_re, z_im))

In [9]:
# numpy matrix
def extend_ZreZim_matrix(dataframe):
    z_re = []
    z_im = []
    for real, img in zip(dataframe['Z_re'].values, dataframe['Z_im']):
        z_re.extend(real)
        z_im.extend(img)
    return np.matrix((z_re, z_im))

In [12]:
data = {}
for collection in rotulo_collection:
    get_collection = conexao.get_mongoDB('levi', collection, limit=10000)
    df = pd.DataFrame(list(get_collection))
    data[collection] = extend_ZreZim_matrix(df)


In [13]:
dataset = tf.data.Dataset.from_tensor_slices(
    (
        data.values(), data.keys()
    )
)

2022-08-30 14:16:49.845774: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-30 14:16:49.845860: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-30 14:16:49.846665: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jherfson-inspiron5452): /proc/driver/nvidia/version does not exist
2022-08-30 14:16:50.226214: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-30 14:16:50.518590: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 28960

In [14]:
for i in dataset.as_numpy_iterator():
    print(i)

(array([[ 3.87869560e+12,  3.76142463e+12,  3.64769931e+12, ...,
         1.38241953e+02,  1.32801976e+02,  1.27576609e+02],
       [-2.81803730e+12, -2.73283496e+12, -2.65020868e+12, ...,
        -1.49810737e+02, -1.43870538e+02, -1.38167217e+02]]), b'RQ_Q')
(array([[ 1.00000000e+01,  1.00000000e+01,  1.00000000e+01, ...,
         6.74760552e+01,  6.38763599e+01,  6.04687003e+01],
       [-1.22603416e-11, -1.26425856e-11, -1.30360461e-11, ...,
        -1.40115459e+02, -1.32640616e+02, -1.25564539e+02]]), b'RQ')
(array([[ 1.99440803e+01,  1.99440803e+01,  1.99440803e+01, ...,
         1.08889833e-11,  9.33943629e-12,  8.01039620e-12],
       [-2.20441774e-09, -2.27314642e-09, -2.34401624e-09, ...,
        -3.31834987e-01, -3.07318596e-01, -2.84613507e-01]]), b'RQRQ_RQ')
(array([[ 3.87869560e+12,  3.76142463e+12,  3.64769931e+12, ...,
         3.49640370e+03,  3.48834867e+03,  3.47905156e+03],
       [-2.81803730e+12, -2.73283496e+12, -2.65020868e+12, ...,
        -2.96199559e+02, -3.18